# Algorithm Analysis

A simple runtime analysis example. Notice that in each version of the algorithm, we count the steps (to empirically verify the different in runtime)

In [2]:
def disjoint1(A, B, C):
    """Return True if there is no element common to all three lists."""
    i=0
    for a in A:
        for b in B:
            for c in C:
                i += 1
                if a == b == c:
                    print('took {} steps'.format(i))
                    return False # we found a common value
    print('took {} steps'.format(i))
    return True

In [3]:
def disjoint2(A, B, C):
    """Return True if there is no element common to all three lists."""
    i=0
    for a in A:
        for b in B:
            i += 1
            if a == b: # only check C if we found match from A and B
                for c in C:
                    i += 1
                    if a == c: # (and thus a == b == c)
                        print('took {} steps'.format(i))
                        return False # we found a common value
    print('took {} steps'.format(i))
    return True # if we reach this, sets are disjoint

`disjoint1` should take $O(n^3)$ steps to run, why?

* the outermost loop is $O(n)$
* for each of the $O(n)$ iterations, another loop is run, which is $O(n)$ (so $O(n^2)$ total)
* for each of $O(n^2)$ iterations, a third loop is run, for a total of $O(n^3)$ iterations

In [4]:
disjoint1({1,2,3,4,5},{6,7,8,9,10},{11,12,13,14,15})
#dis.dis(disjoint1)

took 125 steps


True

`disjoint2` should take $O(n^2)$ steps to run, why?

* the outermost loop is $O(n)$
* for each of the $O(n)$ iterations, another loop is run, which is $O(n)$ (so $O(n^2)$ total)
* Not always will the innermost loop run. It will run at most $n$ times (if $A\subseteq B$ or $B \subseteq A$)
* For each of the (at most) $O(n)$ iterations when $a==b$, a third loop is run, for a total of $O(n^2)$ iterations

Hence, $O(n^2)+O(n^2)$ is $O(n^2)$

All 3 sets disjoint scenerio. Notice the inner loop never executes.

In [5]:
disjoint2({1,2,3,4,5},{6,7,8,9,10},{11,12,13,14,15})

took 25 steps


True

Worst case scenerio. Notice the inner loop executes all $n$ times

In [6]:
disjoint2({1,2,3,4,5},{1,2,3,4,5},{11,12,13,14,15})

took 50 steps


True

Best case scenerio. A common value is immediately found

In [7]:
disjoint2({1,2,3,4,5},{1,2,3,4,5},{1,2,3,4,5})

took 2 steps


False

We can see the low level code using the `dis` module

In [8]:
#import dis
#dis.dis(disjoint1)

# Recursion

In [ ]:
Consider the following file structure